# 2. Interacting with records in databases using yabadaba

This Notebook builds upon the previous one and shows how to interact with records that are stored within databases.

## Adding Records to yabadaba

As mentioned in the previous Notebook, yabadaba by default does not have any Record classes.  You must add Record classes to the recordmanager before the Database operations will work.

In [1]:
import yabadaba

In [2]:
yabadaba.recordmanager.check_styles()

Record styles that passed import:
Record styles that failed import:



Here, I'm importing yabadaba_demo which has definitions for a few Record examples, including FAQ. It also imports yabadaba.recordmanager and incorporates the Record styles into it using the recordmanager.import_styles() method.

In [3]:
import sys

# Append local path to yabadaba_demo - not needed if package is in cwd or is installed
sys.path.append('../')
import yabadaba_demo

In [4]:
yabadaba.recordmanager.check_styles()

Record styles that passed import:
- faq: <class 'yabadaba_demo.record.faq.FAQ.FAQ'>
- demo: <class 'yabadaba_demo.record.demo.Demo.Demo'>
Record styles that failed import:
- bad_record: <class 'ModuleNotFoundError'>: No module named 'package_that_does_not_exist'



Note that one of the Record styles failed import due to a missing package. The recordmanager.import_styles() method allows for such imports to soft-fail rather than hard fail allowing for different modules to have more package requirements than the base code.  

## Specifying a database

The Database objects are initialized by specifying database style and any other parameters necessary to access it.  These access parameters are remembered so that all subsequent database interactions use them without needing to be respecified.

Here, we will simply define a 'local' database consisting of JSON files in a directory on this machine.

In [5]:
database = yabadaba.load_database(style='local', host='./demo_database')
print(database)

database style local at /home/lmh1/Python-packages/yabadaba_demo/doc/demo_database


You can define your database every time, or you can use yabadaba.settings to save the settings for a database under a simple string name for later use.

## Getting records

Records can be retrieved from the database using one of three methods
- __get_records()__ retrieves all matching records and returns them as Record objects.  If you specify "return_df=True", it will also return a pandas.DataFrame containing the metadata fields for the records.
- __get_records_df()__ does the same thing but only returns the pandas.DataFrame.
- __get_record()__ retrieves a single matching record from the database as a Record object.  If no or multiple matches are found, it will throw an error.  This is convenient when your code needs exactly one match. 

In [6]:
database.get_records_df('faq')

,name,question,answer
0,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
1,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
2,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


As both question and answer were defined as 'longstr' values, they have a default 'str_contains' Query associated with them.  The get methods will perform the appropriate database query operations if you specify values for question or answer as parameters.

In [7]:
# Return only records where question contains the word bear.
database.get_records_df('faq', question='bear')

,name,question,answer
0,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.


With get_records, the retrieved Record objects are returned within a numpy array.  This opens up the records to further parsing, which can be done on the associated dataframe then applied to the records.

In [8]:
records, df = database.get_records('faq', return_df=True)

In [9]:
# Use the dataframe to parse out a subset of records
df[df.name.isin(['faq', 'woodchuck'])]

,name,question,answer
0,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
2,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


In [10]:
# Apply the parsing operation directly to the array of records to get the appropriate objects.
for record in records[df.name.isin(['faq', 'woodchuck'])]:
    print(record.name)

faq
woodchuck


## Modifying database records

Record content in the database can then be modified with the following operations
- __add_record()__ adds a new record to the database.  Will throw an error if an existing record of the same name and style already exist.
- __update_record()__ updates the model contents of a record in the database.
- __delete_record()__ removes a record from the database.

For all of these methods, the record can be directly given as an object, or you can specify the record by giving name and style values.  With add and update, you also have to give model contents as well, either in the record object or separately.

In [11]:
# Define a new FAQ record
record = yabadaba.load_record('faq', name='baseball', question="Who's on first?", answer='Yes.')

# Add it to the database
database.add_record(record)
database.get_records_df('faq', refresh_cache=True)

,name,question,answer
0,baseball,Who's on first?,Yes.
1,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
2,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
3,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


In [14]:
# Modify the record and update the model
record.answer = 'Naturally.'
record.build_model()


# Update it in the database
database.update_record(record)
database.get_records_df('faq', refresh_cache=True)

,name,question,answer
0,baseball,Who's on first?,Naturally.
1,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
2,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
3,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


In [15]:
# Delete the record
database.delete_record(record)
database.get_records_df('faq', refresh_cache=True)

,name,question,answer
0,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
1,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
2,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...
